In [1]:
import time
import sys
import numpy as np
import pandas as pd
from sklearn.cross_validation import train_test_split, KFold
import multiprocessing
import os
from tqdm import tqdm
from scipy import sparse

/usr/local/Cellar/python3/3.6.4_2/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import r2_score
from scipy.stats import spearmanr, pearsonr
from sklearn import ensemble

In [3]:
import datetime as dt
from datetime import date
import calendar
from calendar import weekday, day_name

In [4]:
import statsmodels.api as sm
import statsmodels.formula.api as smf
import statsmodels.stats.api as sms
import scipy as sp

/usr/local/Cellar/python3/3.6.4_2/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [5]:
from patsy import dmatrix
import statsmodels.api as sm
import statsmodels.formula.api as smf
import statsmodels.stats.api as sms
import scipy as sp
from sklearn.datasets import make_regression

In [6]:
def get_holidays(fpath):
    
    f = open(fpath)
    lines = f.readlines()
    lines = [line.split(" ")[:3] for line in lines]
    lines = ["{} {} {}".format(line[0], line[1], line[2]) for line in lines]
    lines = pd.to_datetime(lines)
    return pd.DataFrame({"date2":lines})

In [7]:
def get_holiday_names(fpath):
    # holiday_names are holidays + around Black Fridays
    
    f = open(fpath)
    lines = f.readlines()
    lines = [line.strip().split(" ")[:4] for line in lines]
    lines_dt = ["{} {} {}".format(line[0], line[1], line[2]) for line in lines]
    lines_dt = pd.to_datetime(lines_dt)
    lines_hol = [line[3] for line in lines]
    return pd.DataFrame({"date2":lines_dt, "holiday_name":lines_hol})

In [8]:
def preprocess(_df):
    
    df = _df.copy()
    
    # date
    df['date2'] = pd.to_datetime(df['date'])
    
    # weekday
    df['weekday'] = df[['date2']].apply(lambda x: dt.datetime.strftime(x['date2'], '%a'), axis=1)
    # 김태형) 더미변수로 바꿔주기 위해 문자열로 생성

    #df['weekday'] = df.date2.dt.weekday
    df['is_weekend'] = df.date2.dt.weekday.isin([5,6])
    df['is_holiday'] = df.date2.isin(holidays.date2)
    df['is_holiday_weekday'] = df.is_holiday & (df.is_weekend == False)
    df['is_holiday_weekend'] = df.is_holiday &  df.is_weekend

    # bool to int (maybe no meaning)
    df.is_weekend = np.where(df.is_weekend, 1, 0)
    df.is_holiday = np.where(df.is_holiday, 1, 0)
    df.is_holiday_weekday = np.where(df.is_holiday_weekday, 1, 0)
    df.is_holiday_weekend = np.where(df.is_holiday_weekend, 1, 0)
    
    # day, month, year
    df['day'] = df.date2.dt.day
    df['month'] = df.date2.dt.month
    df['year'] = df.date2.dt.year
    
    # around BlackFriday
    df = pd.merge(df, holiday_names, on='date2', how = 'left')
    df.loc[df.holiday_name.isnull(), "holiday_name"] = ""

    around_BlackFriday = ["BlackFridayM3", "BlackFridayM2", "ThanksgivingDay", "BlackFriday",
                          "BlackFriday1", "BlackFriday2", "BlackFriday3"]
    df["around_BlackFriday"] = np.where(df.holiday_name.isin(around_BlackFriday), 
                                        df.holiday_name, "Else")

    return df

In [9]:
train_file = pd.read_csv("tae/train.csv", sep=',')
train_file

date  store_nbr  item_nbr  units
0        2012-01-01          1         1      0
1        2012-01-01          1         2      0
2        2012-01-01          1         3      0
3        2012-01-01          1         4      0
4        2012-01-01          1         5      0
5        2012-01-01          1         6      0
6        2012-01-01          1         7      0
7        2012-01-01          1         8      0
8        2012-01-01          1         9     29
9        2012-01-01          1        10      0
10       2012-01-01          1        11      0
11       2012-01-01          1        12      0
12       2012-01-01          1        13      0
13       2012-01-01          1        14      0
14       2012-01-01          1        15      0
15       2012-01-01          1        16      0
16       2012-01-01          1        17      0
17       2012-01-01          1        18      0
18       2012-01-01          1        19      0
19       2012-01-01          1        20      0
20       2012-01-01          1        21      0
21       2012-01-01          1        22      0
22       2012-01-01          1        23      0
23       2012-01-01          1        24      0
24       2012-01-01          1        25      0
25       2012-01-01          1        26      0
26       2012-01-01          1        27      0
27       2012-01-01          1        28      2
28       2012-01-01          1        29      0
29       2012-01-01          1        30      0
...             ...        ...       ...    ...
4617570  2014-10-31         45        82      0
4617571  2014-10-31         45        83      0
4617572  2014-10-31         45        84      0
4617573  2014-10-31         45        85      0
4617574  2014-10-31         45        86      0
4617575  2014-10-31         45        87      0
4617576  2014-10-31         45        88      0
4617577  2014-10-31         45        89      0
4617578  2014-10-31         45        90      0
4617579  2014-10-31         45        91      0
4617580  2014-10-31         45        92      0
4617581  2014-10-31         45        93      0
4617582  2014-10-31         45        94      0
4617583  2014-10-31         45        95      0
4617584  2014-10-31         45        96      0
4617585  2014-10-31         45        97      0
4617586  2014-10-31         45        98      0
4617587  2014-10-31         45        99      0
4617588  2014-10-31         45       100      0
4617589  2014-10-31         45       101      0
4617590  2014-10-31         45       102      0
4617591  2014-10-31         45       103      0
4617592  2014-10-31         45       104      0
4617593  2014-10-31         45       105      0
4617594  2014-10-31         45       106      0
4617595  2014-10-31         45       107      0
4617596  2014-10-31         45       108      0
4617597  2014-10-31         45       109      0
4617598  2014-10-31         45       110      0
4617599  2014-10-31         45       111      0

[4617600 rows x 4 columns]

In [15]:
train_file = preprocess(train_file)
train_file

date  store_nbr  item_nbr  units      date2 weekday  \
0        2012-01-01          1         1      0 2012-01-01     Sun   
1        2012-01-01          1         2      0 2012-01-01     Sun   
2        2012-01-01          1         3      0 2012-01-01     Sun   
3        2012-01-01          1         4      0 2012-01-01     Sun   
4        2012-01-01          1         5      0 2012-01-01     Sun   
5        2012-01-01          1         6      0 2012-01-01     Sun   
6        2012-01-01          1         7      0 2012-01-01     Sun   
7        2012-01-01          1         8      0 2012-01-01     Sun   
8        2012-01-01          1         9     29 2012-01-01     Sun   
9        2012-01-01          1        10      0 2012-01-01     Sun   
10       2012-01-01          1        11      0 2012-01-01     Sun   
11       2012-01-01          1        12      0 2012-01-01     Sun   
12       2012-01-01          1        13      0 2012-01-01     Sun   
13       2012-01-01          1        14      0 2012-01-01     Sun   
14       2012-01-01          1        15      0 2012-01-01     Sun   
15       2012-01-01          1        16      0 2012-01-01     Sun   
16       2012-01-01          1        17      0 2012-01-01     Sun   
17       2012-01-01          1        18      0 2012-01-01     Sun   
18       2012-01-01          1        19      0 2012-01-01     Sun   
19       2012-01-01          1        20      0 2012-01-01     Sun   
20       2012-01-01          1        21      0 2012-01-01     Sun   
21       2012-01-01          1        22      0 2012-01-01     Sun   
22       2012-01-01          1        23      0 2012-01-01     Sun   
23       2012-01-01          1        24      0 2012-01-01     Sun   
24       2012-01-01          1        25      0 2012-01-01     Sun   
25       2012-01-01          1        26      0 2012-01-01     Sun   
26       2012-01-01          1        27      0 2012-01-01     Sun   
27       2012-01-01          1        28      2 2012-01-01     Sun   
28       2012-01-01          1        29      0 2012-01-01     Sun   
29       2012-01-01          1        30      0 2012-01-01     Sun   
...             ...        ...       ...    ...        ...     ...   
4617570  2014-10-31         45        82      0 2014-10-31     Fri   
4617571  2014-10-31         45        83      0 2014-10-31     Fri   
4617572  2014-10-31         45        84      0 2014-10-31     Fri   
4617573  2014-10-31         45        85      0 2014-10-31     Fri   
4617574  2014-10-31         45        86      0 2014-10-31     Fri   
4617575  2014-10-31         45        87      0 2014-10-31     Fri   
4617576  2014-10-31         45        88      0 2014-10-31     Fri   
4617577  2014-10-31         45        89      0 2014-10-31     Fri   
4617578  2014-10-31         45        90      0 2014-10-31     Fri   
4617579  2014-10-31         45        91      0 2014-10-31     Fri   
4617580  2014-10-31         45        92      0 2014-10-31     Fri   
4617581  2014-10-31         45        93      0 2014-10-31     Fri   
4617582  2014-10-31         45        94      0 2014-10-31     Fri   
4617583  2014-10-31         45        95      0 2014-10-31     Fri   
4617584  2014-10-31         45        96      0 2014-10-31     Fri   
4617585  2014-10-31         45        97      0 2014-10-31     Fri   
4617586  2014-10-31         45        98      0 2014-10-31     Fri   
4617587  2014-10-31         45        99      0 2014-10-31     Fri   
4617588  2014-10-31         45       100      0 2014-10-31     Fri   
4617589  2014-10-31         45       101      0 2014-10-31     Fri   
4617590  2014-10-31         45       102      0 2014-10-31     Fri   
4617591  2014-10-31         45       103      0 2014-10-31     Fri   
4617592  2014-10-31         45       104      0 2014-10-31     Fri   
4617593  2014-10-31         45       105      0 2014-10-31     Fri   
4617594  2014-10-31         45       106      0 2014-10-31     Fri   
4617595  2014-10-31      

In [11]:
key_file = pd.read_csv("tae/key.csv", sep=',')
key_file

store_nbr  station_nbr
0           1            1
1           2           14
2           3            7
3           4            9
4           5           12
5           6           14
6           7            6
7           8            4
8           9           17
9          10           12
10         11           10
11         12           11
12         13            6
13         14           16
14         15           13
15         16            2
16         17           20
17         18           17
18         19           15
19         20            7
20         21            3
21         22           10
22         23           17
23         24            9
24         25           13
25         26           17
26         27           10
27         28            7
28         29            3
29         30           19
30         31           17
31         32           13
32         33            3
33         34           17
34         35            5
35         36           18
36         37           13
37         38           14
38         39            8
39         40           13
40         41           12
41         42           14
42         43           11
43         44           12
44         45           16

In [14]:
holidays = get_holidays("tae/holidays.txt")
holiday_names = get_holiday_names("tae/holiday_names.txt")

In [16]:
result = pd.merge(train_file, key_file, left_on='store_nbr', right_on='store_nbr', how='outer')
result

date  store_nbr  item_nbr  units      date2 weekday  \
0        2012-01-01          1         1      0 2012-01-01     Sun   
1        2012-01-01          1         2      0 2012-01-01     Sun   
2        2012-01-01          1         3      0 2012-01-01     Sun   
3        2012-01-01          1         4      0 2012-01-01     Sun   
4        2012-01-01          1         5      0 2012-01-01     Sun   
5        2012-01-01          1         6      0 2012-01-01     Sun   
6        2012-01-01          1         7      0 2012-01-01     Sun   
7        2012-01-01          1         8      0 2012-01-01     Sun   
8        2012-01-01          1         9     29 2012-01-01     Sun   
9        2012-01-01          1        10      0 2012-01-01     Sun   
10       2012-01-01          1        11      0 2012-01-01     Sun   
11       2012-01-01          1        12      0 2012-01-01     Sun   
12       2012-01-01          1        13      0 2012-01-01     Sun   
13       2012-01-01          1        14      0 2012-01-01     Sun   
14       2012-01-01          1        15      0 2012-01-01     Sun   
15       2012-01-01          1        16      0 2012-01-01     Sun   
16       2012-01-01          1        17      0 2012-01-01     Sun   
17       2012-01-01          1        18      0 2012-01-01     Sun   
18       2012-01-01          1        19      0 2012-01-01     Sun   
19       2012-01-01          1        20      0 2012-01-01     Sun   
20       2012-01-01          1        21      0 2012-01-01     Sun   
21       2012-01-01          1        22      0 2012-01-01     Sun   
22       2012-01-01          1        23      0 2012-01-01     Sun   
23       2012-01-01          1        24      0 2012-01-01     Sun   
24       2012-01-01          1        25      0 2012-01-01     Sun   
25       2012-01-01          1        26      0 2012-01-01     Sun   
26       2012-01-01          1        27      0 2012-01-01     Sun   
27       2012-01-01          1        28      2 2012-01-01     Sun   
28       2012-01-01          1        29      0 2012-01-01     Sun   
29       2012-01-01          1        30      0 2012-01-01     Sun   
...             ...        ...       ...    ...        ...     ...   
4617570  2014-10-31         35        82      0 2014-10-31     Fri   
4617571  2014-10-31         35        83      0 2014-10-31     Fri   
4617572  2014-10-31         35        84      0 2014-10-31     Fri   
4617573  2014-10-31         35        85      0 2014-10-31     Fri   
4617574  2014-10-31         35        86      0 2014-10-31     Fri   
4617575  2014-10-31         35        87      0 2014-10-31     Fri   
4617576  2014-10-31         35        88      0 2014-10-31     Fri   
4617577  2014-10-31         35        89      0 2014-10-31     Fri   
4617578  2014-10-31         35        90      0 2014-10-31     Fri   
4617579  2014-10-31         35        91      0 2014-10-31     Fri   
4617580  2014-10-31         35        92      0 2014-10-31     Fri   
4617581  2014-10-31         35        93      0 2014-10-31     Fri   
4617582  2014-10-31         35        94      0 2014-10-31     Fri   
4617583  2014-10-31         35        95      0 2014-10-31     Fri   
4617584  2014-10-31         35        96      0 2014-10-31     Fri   
4617585  2014-10-31         35        97      0 2014-10-31     Fri   
4617586  2014-10-31         35        98      0 2014-10-31     Fri   
4617587  2014-10-31         35        99      0 2014-10-31     Fri   
4617588  2014-10-31         35       100      0 2014-10-31     Fri   
4617589  2014-10-31         35       101      0 2014-10-31     Fri   
4617590  2014-10-31         35       102      0 2014-10-31     Fri   
4617591  2014-10-31         35       103      0 2014-10-31     Fri   
4617592  2014-10-31         35       104      0 2014-10-31     Fri   
4617593  2014-10-31         35       105      0 2014-10-31     Fri   
4617594  2014-10-31         35       106      0 2014-10-31     Fri   
4617595  2014-10-31      

In [18]:
weather_file = pd.read_csv("tae/weather_processed.csv", sep = ',')
#weather_file['date'] = pd.to_datetime(weather_file['date'], errors='coerce')
weather_file

station_nbr        date  tmax  tmin  tavg  depart  dewpoint  wetbulb  \
0                1  2012-01-01  52.0  31.0  42.0    16.0      36.0     40.0   
1                2  2012-01-01  48.0  33.0  41.0    16.0      37.0     39.0   
2                3  2012-01-01  55.0  34.0  45.0     9.0      24.0     36.0   
3                4  2012-01-01  63.0  47.0  55.0     4.0      28.0     43.0   
4                6  2012-01-01  63.0  34.0  49.0     0.0      31.0     43.0   
5                7  2012-01-01  50.0  33.0  42.0     0.0      26.0     35.0   
6                8  2012-01-01  66.0  45.0  42.0     0.0      34.0     46.0   
7                9  2012-01-01  34.0  19.0  27.0     0.0      17.0     23.0   
8               10  2012-01-01  73.0  53.0  63.0     0.0      55.0     58.0   
9               11  2012-01-01  72.0  48.0  60.0     7.0      54.0     56.0   
10              12  2012-01-01  72.0  48.0  60.0     7.0      54.0     57.0   
11              13  2012-01-01  38.0  18.0  28.0     7.0      12.0     21.0   
12              14  2012-01-01  50.0  34.0  42.0     5.0      25.0     35.0   
13              15  2012-01-01  48.0  26.0  37.0    16.0      35.0     38.0   
14              16  2012-01-01  50.0  30.0  40.0    16.0      37.0     40.0   
15              17  2012-01-01  56.0  38.0  47.0    16.0      23.0     37.0   
16              18  2012-01-01  59.0  40.0  50.0     4.0      28.0     40.0   
17              19  2012-01-01  38.0  25.0  32.0    10.0      26.0     30.0   
18              20  2012-01-01  59.0  41.0  50.0    10.0      28.0     40.0   
19               1  2012-01-02  50.0  31.0  41.0    10.0      26.0     35.0   
20               2  2012-01-02  46.0  28.0  37.0    12.0      24.0     32.0   
21               3  2012-01-02  45.0  24.0  35.0    -1.0      11.0     28.0   
22               4  2012-01-02  60.0  37.0  49.0    -2.0      26.0     39.0   
23               6  2012-01-02  60.0  30.0  45.0    -4.0      27.0     38.0   
24               7  2012-01-02  44.0  26.0  35.0    -4.0      17.0     29.0   
25               8  2012-01-02  59.0  36.0  35.0    -4.0      29.0     40.0   
26               9  2012-01-02  21.0  11.0  16.0    -4.0       8.0     15.0   
27              10  2012-01-02  65.0  45.0  55.0    -4.0      33.0     46.0   
28              11  2012-01-02  63.0  40.0  52.0    -1.0      31.0     44.0   
29              12  2012-01-02  65.0  47.0  56.0    -1.0      33.0     46.0   
...            ...         ...   ...   ...   ...     ...       ...      ...   
20487           11  2014-10-30  81.0  59.0  70.0     4.0      63.0     65.0   
20488           12  2014-10-30  81.0  61.0  71.0     4.0      63.0     66.0   
20489           13  2014-10-30  61.0  32.0  47.0     4.0      26.0     38.0   
20490           14  2014-10-30  77.0  45.0  61.0     5.0      39.0     50.0   
20491           15  2014-10-30  56.0  30.0  43.0     0.0      36.0     39.0   
20492           16  2014-10-30  57.0  34.0  46.0     0.0      39.0     43.0   
20493           17  2014-10-30  77.0  43.0  60.0     0.0      41.0     50.0   
20494           18  2014-10-30  79.0  56.0  68.0     5.0      51.0     58.0   
20495           19  2014-10-30  51.0  32.0  42.0    -4.0      33.0     40.0   
20496           20  2014-10-30  80.0  52.0  66.0    -4.0      50.0     57.0   
20497            1  2014-10-31  53.0  40.0  47.0    -4.0      36.0     41.0   
20498            2  2014-10-31  49.0  40.0  45.0     0.0      35.0     40.0   
20499            3  2014-10-31  55.0  33.0  44.0   -12.0      32.0     41.0   
20500            4  2014-10-31  76.0  56.0  66.0    -1.0      47.0     56.0   
20501            5  2014-10-31  76.0  56.0  66.0    -1.0      37.0     56.0   
20502            6  2014-10-31  76.0  51.0  64.0     0.0      47.0     55.0   
20503            7  2014-10-31  58.0  38.0  48.0     0.0      32.0     41.0   
20504            8  2014-10-31  78.0  55.0  67.0     0.0      32.0     41.0   
20505            9  2014-10-31  39.0  25.0  32.0     0.0 

In [19]:
all_data_result = pd.merge(result, weather_file, on=['date', 'station_nbr'], how='left')
all_data_result

date  store_nbr  item_nbr  units      date2 weekday  \
0        2012-01-01          1         1      0 2012-01-01     Sun   
1        2012-01-01          1         2      0 2012-01-01     Sun   
2        2012-01-01          1         3      0 2012-01-01     Sun   
3        2012-01-01          1         4      0 2012-01-01     Sun   
4        2012-01-01          1         5      0 2012-01-01     Sun   
5        2012-01-01          1         6      0 2012-01-01     Sun   
6        2012-01-01          1         7      0 2012-01-01     Sun   
7        2012-01-01          1         8      0 2012-01-01     Sun   
8        2012-01-01          1         9     29 2012-01-01     Sun   
9        2012-01-01          1        10      0 2012-01-01     Sun   
10       2012-01-01          1        11      0 2012-01-01     Sun   
11       2012-01-01          1        12      0 2012-01-01     Sun   
12       2012-01-01          1        13      0 2012-01-01     Sun   
13       2012-01-01          1        14      0 2012-01-01     Sun   
14       2012-01-01          1        15      0 2012-01-01     Sun   
15       2012-01-01          1        16      0 2012-01-01     Sun   
16       2012-01-01          1        17      0 2012-01-01     Sun   
17       2012-01-01          1        18      0 2012-01-01     Sun   
18       2012-01-01          1        19      0 2012-01-01     Sun   
19       2012-01-01          1        20      0 2012-01-01     Sun   
20       2012-01-01          1        21      0 2012-01-01     Sun   
21       2012-01-01          1        22      0 2012-01-01     Sun   
22       2012-01-01          1        23      0 2012-01-01     Sun   
23       2012-01-01          1        24      0 2012-01-01     Sun   
24       2012-01-01          1        25      0 2012-01-01     Sun   
25       2012-01-01          1        26      0 2012-01-01     Sun   
26       2012-01-01          1        27      0 2012-01-01     Sun   
27       2012-01-01          1        28      2 2012-01-01     Sun   
28       2012-01-01          1        29      0 2012-01-01     Sun   
29       2012-01-01          1        30      0 2012-01-01     Sun   
...             ...        ...       ...    ...        ...     ...   
4617570  2014-10-31         35        82      0 2014-10-31     Fri   
4617571  2014-10-31         35        83      0 2014-10-31     Fri   
4617572  2014-10-31         35        84      0 2014-10-31     Fri   
4617573  2014-10-31         35        85      0 2014-10-31     Fri   
4617574  2014-10-31         35        86      0 2014-10-31     Fri   
4617575  2014-10-31         35        87      0 2014-10-31     Fri   
4617576  2014-10-31         35        88      0 2014-10-31     Fri   
4617577  2014-10-31         35        89      0 2014-10-31     Fri   
4617578  2014-10-31         35        90      0 2014-10-31     Fri   
4617579  2014-10-31         35        91      0 2014-10-31     Fri   
4617580  2014-10-31         35        92      0 2014-10-31     Fri   
4617581  2014-10-31         35        93      0 2014-10-31     Fri   
4617582  2014-10-31         35        94      0 2014-10-31     Fri   
4617583  2014-10-31         35        95      0 2014-10-31     Fri   
4617584  2014-10-31         35        96      0 2014-10-31     Fri   
4617585  2014-10-31         35        97      0 2014-10-31     Fri   
4617586  2014-10-31         35        98      0 2014-10-31     Fri   
4617587  2014-10-31         35        99      0 2014-10-31     Fri   
4617588  2014-10-31         35       100      0 2014-10-31     Fri   
4617589  2014-10-31         35       101      0 2014-10-31     Fri   
4617590  2014-10-31         35       102      0 2014-10-31     Fri   
4617591  2014-10-31         35       103      0 2014-10-31     Fri   
4617592  2014-10-31         35       104      0 2014-10-31     Fri   
4617593  2014-10-31         35       105      0 2014-10-31     Fri   
4617594  2014-10-31         35       106      0 2014-10-31     Fri   
4617595  2014-10-31      

In [6]:
#여기서 부터
all_data_result = pd.read_csv('another_all_train_matrix.csv', sep=',')

In [20]:
all_data_result

date  store_nbr  item_nbr  units      date2 weekday  \
0        2012-01-01          1         1      0 2012-01-01     Sun   
1        2012-01-01          1         2      0 2012-01-01     Sun   
2        2012-01-01          1         3      0 2012-01-01     Sun   
3        2012-01-01          1         4      0 2012-01-01     Sun   
4        2012-01-01          1         5      0 2012-01-01     Sun   
5        2012-01-01          1         6      0 2012-01-01     Sun   
6        2012-01-01          1         7      0 2012-01-01     Sun   
7        2012-01-01          1         8      0 2012-01-01     Sun   
8        2012-01-01          1         9     29 2012-01-01     Sun   
9        2012-01-01          1        10      0 2012-01-01     Sun   
10       2012-01-01          1        11      0 2012-01-01     Sun   
11       2012-01-01          1        12      0 2012-01-01     Sun   
12       2012-01-01          1        13      0 2012-01-01     Sun   
13       2012-01-01          1        14      0 2012-01-01     Sun   
14       2012-01-01          1        15      0 2012-01-01     Sun   
15       2012-01-01          1        16      0 2012-01-01     Sun   
16       2012-01-01          1        17      0 2012-01-01     Sun   
17       2012-01-01          1        18      0 2012-01-01     Sun   
18       2012-01-01          1        19      0 2012-01-01     Sun   
19       2012-01-01          1        20      0 2012-01-01     Sun   
20       2012-01-01          1        21      0 2012-01-01     Sun   
21       2012-01-01          1        22      0 2012-01-01     Sun   
22       2012-01-01          1        23      0 2012-01-01     Sun   
23       2012-01-01          1        24      0 2012-01-01     Sun   
24       2012-01-01          1        25      0 2012-01-01     Sun   
25       2012-01-01          1        26      0 2012-01-01     Sun   
26       2012-01-01          1        27      0 2012-01-01     Sun   
27       2012-01-01          1        28      2 2012-01-01     Sun   
28       2012-01-01          1        29      0 2012-01-01     Sun   
29       2012-01-01          1        30      0 2012-01-01     Sun   
...             ...        ...       ...    ...        ...     ...   
4617570  2014-10-31         35        82      0 2014-10-31     Fri   
4617571  2014-10-31         35        83      0 2014-10-31     Fri   
4617572  2014-10-31         35        84      0 2014-10-31     Fri   
4617573  2014-10-31         35        85      0 2014-10-31     Fri   
4617574  2014-10-31         35        86      0 2014-10-31     Fri   
4617575  2014-10-31         35        87      0 2014-10-31     Fri   
4617576  2014-10-31         35        88      0 2014-10-31     Fri   
4617577  2014-10-31         35        89      0 2014-10-31     Fri   
4617578  2014-10-31         35        90      0 2014-10-31     Fri   
4617579  2014-10-31         35        91      0 2014-10-31     Fri   
4617580  2014-10-31         35        92      0 2014-10-31     Fri   
4617581  2014-10-31         35        93      0 2014-10-31     Fri   
4617582  2014-10-31         35        94      0 2014-10-31     Fri   
4617583  2014-10-31         35        95      0 2014-10-31     Fri   
4617584  2014-10-31         35        96      0 2014-10-31     Fri   
4617585  2014-10-31         35        97      0 2014-10-31     Fri   
4617586  2014-10-31         35        98      0 2014-10-31     Fri   
4617587  2014-10-31         35        99      0 2014-10-31     Fri   
4617588  2014-10-31         35       100      0 2014-10-31     Fri   
4617589  2014-10-31         35       101      0 2014-10-31     Fri   
4617590  2014-10-31         35       102      0 2014-10-31     Fri   
4617591  2014-10-31         35       103      0 2014-10-31     Fri   
4617592  2014-10-31         35       104      0 2014-10-31     Fri   
4617593  2014-10-31         35       105      0 2014-10-31     Fri   
4617594  2014-10-31         35       106      0 2014-10-31     Fri   
4617595  2014-10-31      

In [21]:
ls_store_nbr = set(all_data_result.store_nbr)
mod = sys.modules[__name__]
df = pd.DataFrame()
for store_nbr in tqdm(ls_store_nbr):
    #setattr(mod, 'store_{}'.format(store_nbr), all_data_result[all_data_result['store_nbr'] == store_nbr])
    data = all_data_result[all_data_result['store_nbr'] == store_nbr]
    mergedata = pd.merge(data, weather_file, on = ['date'])
    #df = df.append(mergedata, ignore_index=True)
    setattr(mod, 'store_{}_data'.format(store_nbr), mergedata)
    #mergedata.to_csv("store_{}_data.csv".format(store_nbr), sep=',', index=None)

100%|██████████| 45/45 [01:06<00:00,  1.48s/it]


In [28]:
def store_storage(result, store) :
    """
    회귀분석 결과 모음

    전역변수로 아래 데이터 프레임 선언
    result_sets = pd.DataFrame(columns = ["R square", "AIC", "BIC", "P_fstatics", "P_omnibus", "P_jb", "Cond_no"])

    Variables_name.fit() 상태로 삽입
    ex)result = model.fit() 이면
    result를 삽입    


    reason = 이전 데이터 셋에서 무엇을 변화시켰는지 적을 것.
    """    
    
    global result_sets
    result_sets = pd.DataFrame(columns = ["store_nbr", "R square", "F_statics", "Prob(F_statics)", "AIC", "BIC",
                                          "Omnibus", "P_omnibus", "P_jb", "Cond_no"])
                
    model = sm.OLS.from_formula( "units ~ C(item_nbr) + C(weekday) +"
                                 "is_weekend + is_holiday_weekday + "
                                 "is_holiday_weekend + "
                                 "tmax + tmin + tavg + depart + dewpoint + wetbulb + heat + cool + sunrise + "
                                 "sunset + snowfall + preciptotal + stnpressure + sealevel + resultspeed + resultdir +"
                                 "avgspeed", data=result)

    result = model.fit()

    summary = result.summary()
    
    t1 = summary.tables[0]
    t2 = summary.tables[1]
    t3 = summary.tables[2]
    
    r_squared = float(t1.data[0][3])
    fstatics = float(t1.data[2][3])
    p_fstatics = float(t1.data[3][3])
    AIC = float(t1.data[5][3])
    BIC = float(t1.data[6][3])
    

    omnibus = float(t3.data[0][3])
    p_omnibus = float(t3.data[1][1])
    p_jb = float(t3.data[2][3])
    con = float(t3.data[3][3])

    put = {
        "store_nbr" : store,
        "R square" : r_squared,
        "F_statics" : fstatics,
        "Prob(F_statics)" : p_fstatics,
        "AIC" : AIC,
        "BIC" : BIC,
        "Omnibus" : omnibus,
        "P_omnibus" : p_omnibus,
        "P_jb" : p_jb,
        "Cond_no" : con,  
    }
    
    result_sets.loc[len(result_sets)] = put
    
    for i in range(2, len(t2)):
        result_sets = result_sets.append({
            "Name" : t2.data[i][0],
            "std err" : t2.data[i][2],
            "t" : t2.data[i][3],
            "P>|t|" : t2.data[i][4],
            "0.025" : t2.data[i][5],
            "0.975" : t2.data[i][6],
        }, ignore_index=True)
 
    return result_sets

In [25]:
def item_storage(result, item) :
    """
    회귀분석 결과 모음

    전역변수로 아래 데이터 프레임 선언
    result_sets = pd.DataFrame(columns = ["R square", "AIC", "BIC", "P_fstatics", "P_omnibus", "P_jb", "Cond_no"])

    Variables_name.fit() 상태로 삽입
    ex)result = model.fit() 이면
    result를 삽입    


    reason = 이전 데이터 셋에서 무엇을 변화시켰는지 적을 것.
    """    
    
    global result_sets
    result_sets = pd.DataFrame(columns = ["store_nbr", "R square", "F_statics", "Prob(F_statics)", "AIC", "BIC",
                                          "Omnibus", "P_omnibus", "P_jb", "Cond_no"])
                
    model = sm.OLS.from_formula( "units ~ C(weekday) +"
                                 "is_weekend + is_holiday_weekday + "
                                 "is_holiday_weekend + "
                                 "tmax + tmin + tavg + depart + dewpoint + wetbulb + heat + cool + sunrise + "
                                 "sunset + snowfall + preciptotal + stnpressure + sealevel + resultspeed + resultdir +"
                                 "avgspeed", data=result)

    result = model.fit()

    summary = result.summary()
    
    t1 = summary.tables[0]
    t2 = summary.tables[1]
    t3 = summary.tables[2]
    
    r_squared = float(t1.data[0][3])
    fstatics = float(t1.data[2][3])
    p_fstatics = float(t1.data[3][3])
    AIC = float(t1.data[5][3])
    BIC = float(t1.data[6][3])
    

    omnibus = float(t3.data[0][3])
    p_omnibus = float(t3.data[1][1])
    p_jb = float(t3.data[2][3])
    con = float(t3.data[3][3])

    put = {
        "item_nbr" : item,
        "R square" : r_squared,
        "F_statics" : fstatics,
        "Prob(F_statics)" : p_fstatics,
        "AIC" : AIC,
        "BIC" : BIC,
        "Omnibus" : omnibus,
        "P_omnibus" : p_omnibus,
        "P_jb" : p_jb,
        "Cond_no" : con,  
    }
    
    result_sets.loc[len(result_sets)] = put
    
    for i in range(2, len(t2)):
        result_sets = result_sets.append({
            "Name" : t2.data[i][0],
            "std err" : t2.data[i][2],
            "t" : t2.data[i][3],
            "P>|t|" : t2.data[i][4],
            "0.025" : t2.data[i][5],
            "0.975" : t2.data[i][6],
        }, ignore_index=True)
 
    return result_sets

In [23]:
#store_nbr 기준으로 OLS 분류
ls_store_nbr = set(all_data_result.store_nbr)
df = pd.DataFrame()
for store_nbr in tqdm(ls_store_nbr):
    data = all_data_result[all_data_result['store_nbr'] == store_nbr]
    #data = data[data['units'] > 0]
    tmp_df = storage(data, store_nbr)
    df = df.append(tmp_df, ignore_index=True)

100%|██████████| 45/45 [00:38<00:00,  1.16it/s]


In [23]:
df.to_csv("model_OLS_2.csv", sep=',', index=None)

In [26]:
#item_nbr 기준으로 OLS 분류
ls_item_nbr = set(all_data_result.item_nbr)
df_item = pd.DataFrame()
for item_nbr in tqdm(ls_item_nbr):
    data = all_data_result[all_data_result['item_nbr'] == item_nbr]
    data = data[data['units'] > 0]
    tmp_df = item_storage(data, item_nbr)
    df_item = df_item.append(tmp_df, ignore_index=True)

100%|██████████| 111/111 [00:17<00:00,  6.41it/s]


In [27]:
df_item.to_csv("model_item_OLS.csv", sep=',', index=None)

In [ ]:
df_item.to_csv("model_item_units_OLS.csv", sep=',', index=None)

In [37]:
def get_features(data, start_num=0, end_num=None, scale=False):
    features = list(data.columns)[start_num:end_num]
    feature_n = len(features)

    if scale:
        features = list(map(lambda x: "scale({})".format(x), features))
        features = " + ".join(features)

    else:
        features = " + ".join(features)

    return features

a = str(get_features(all_data_result, 0))
a

'date + store_nbr + item_nbr + units + station_nbr + day + holiday_name + year + month + weekday + tmax + tmin + tavg + depart + dewpoint + wetbulb + heat + cool + sunrise + sunset + snowfall + preciptotal + stnpressure + sealevel + resultspeed + resultdir + avgspeed'